In [50]:
import pandas as pd
import numpy as np
messages =pd.read_csv('SMSSpamCollection.txt', sep='\t', header=None, names=['label', 'message'])

import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [51]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash pound txt csh send cost p day day tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw'

In [52]:
#simple_preprocess library can also be used for text preprocessing

#simple preprocess is used to convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.


from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
words = []
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for word in sent_token:
        words.append(simple_preprocess(word))
        
    

In [53]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  'receivea',
 

In [54]:
#lets train word2vec model now from scratch 
import gensim
from gensim.models import Word2Vec

model = Word2Vec(sentences=words, vector_size=100, window=5) #min_count=1 means words with frequency less than 1 will be ignored

In [55]:
#to get all vocabulary words
vocab = model.wv.index_to_key
vocab

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'day',
 'ok',
 'free',
 'know',
 'come',
 'like',
 'good',
 'time',
 'got',
 'love',
 'text',
 'want',
 'send',
 'one',
 'need',
 'txt',
 'today',
 'going',
 'stop',
 'home',
 'lor',
 'sorry',
 'see',
 'still',
 'mobile',
 'take',
 'back',
 'da',
 'reply',
 'dont',
 'think',
 'tell',
 'week',
 'phone',
 'hi',
 'new',
 'later',
 'please',
 'pls',
 'co',
 'msg',
 'min',
 'dear',
 'night',
 'make',
 'message',
 'well',
 'say',
 'thing',
 'much',
 'hope',
 'oh',
 'claim',
 'great',
 'hey',
 'number',
 'give',
 'happy',
 'work',
 'friend',
 'wat',
 'way',
 'yes',
 'www',
 'let',
 'prize',
 'right',
 'tomorrow',
 'already',
 'tone',
 'said',
 'ask',
 'win',
 'amp',
 'cash',
 'life',
 'im',
 'yeah',
 'really',
 'babe',
 'meet',
 'find',
 'miss',
 'morning',
 'last',
 'service',
 'year',
 'uk',
 'thanks',
 'care',
 'would',
 'anything',
 'com',
 'also',
 'nokia',
 'lol',
 'every',
 'feel',
 'keep',
 'pick',
 'sure',
 'sent',
 'contact',
 'urgent',


In [56]:
#vcocab size
len(vocab)
model.corpus_count

5564

In [57]:
#epochs is number of iterations over corpus that is equal to window size
model.epochs

5

In [58]:
model.wv.similar_by_word('good')

[('day', 0.9995916485786438),
 ('give', 0.9995778203010559),
 ('thing', 0.9995761513710022),
 ('much', 0.9995620250701904),
 ('keep', 0.9995589256286621),
 ('thanks', 0.9995514154434204),
 ('today', 0.9995461106300354),
 ('one', 0.9995459318161011),
 ('come', 0.9995437264442444),
 ('said', 0.9995428323745728)]

In [59]:
#average word2vec for a sentence
def avg_word2vec(sentence):
    vectors = [model.wv[word] for word in sentence if word in model.wv.index_to_key]
    if len(vectors) == 0:
        return np.zeros(100)
    return np.mean(vectors, axis=0)



In [60]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:
from tqdm import tqdm
#apply word2vec to entire corpus
X=[]
for i in tqdm(range(len(words))):
    print("hello", i)  #to check progress
    X.append(avg_word2vec(words[i]))

 25%|██▍       | 1390/5564 [00:00<00:00, 13898.03it/s]

hello 0
hello 1
hello 2
hello 3
hello 4
hello 5
hello 6
hello 7
hello 8
hello 9
hello 10
hello 11
hello 12
hello 13
hello 14
hello 15
hello 16
hello 17
hello 18
hello 19
hello 20
hello 21
hello 22
hello 23
hello 24
hello 25
hello 26
hello 27
hello 28
hello 29
hello 30
hello 31
hello 32
hello 33
hello 34
hello 35
hello 36
hello 37
hello 38
hello 39
hello 40
hello 41
hello 42
hello 43
hello 44
hello 45
hello 46
hello 47
hello 48
hello 49
hello 50
hello 51
hello 52
hello 53
hello 54
hello 55
hello 56
hello 57
hello 58
hello 59
hello 60
hello 61
hello 62
hello 63
hello 64
hello 65
hello 66
hello 67
hello 68
hello 69
hello 70
hello 71
hello 72
hello 73
hello 74
hello 75
hello 76
hello 77
hello 78
hello 79
hello 80
hello 81
hello 82
hello 83
hello 84
hello 85
hello 86
hello 87
hello 88
hello 89
hello 90
hello 91
hello 92
hello 93
hello 94
hello 95
hello 96
hello 97
hello 98
hello 99
hello 100
hello 101
hello 102
hello 103
hello 104
hello 105
hello 106
hello 107
hello 108
hello 109
hello 110


 50%|█████     | 2805/5564 [00:00<00:00, 14041.26it/s]

hello 2609
hello 2610
hello 2611
hello 2612
hello 2613
hello 2614
hello 2615
hello 2616
hello 2617
hello 2618
hello 2619
hello 2620
hello 2621
hello 2622
hello 2623
hello 2624
hello 2625
hello 2626
hello 2627
hello 2628
hello 2629
hello 2630
hello 2631
hello 2632
hello 2633
hello 2634
hello 2635
hello 2636
hello 2637
hello 2638
hello 2639
hello 2640
hello 2641
hello 2642
hello 2643
hello 2644
hello 2645
hello 2646
hello 2647
hello 2648
hello 2649
hello 2650
hello 2651
hello 2652
hello 2653
hello 2654
hello 2655
hello 2656
hello 2657
hello 2658
hello 2659
hello 2660
hello 2661
hello 2662
hello 2663
hello 2664
hello 2665
hello 2666
hello 2667
hello 2668
hello 2669
hello 2670
hello 2671
hello 2672
hello 2673
hello 2674
hello 2675
hello 2676
hello 2677
hello 2678
hello 2679
hello 2680
hello 2681
hello 2682
hello 2683
hello 2684
hello 2685
hello 2686
hello 2687
hello 2688
hello 2689
hello 2690
hello 2691
hello 2692
hello 2693
hello 2694
hello 2695
hello 2696
hello 2697
hello 2698
hello 2699

 76%|███████▌  | 4242/5564 [00:00<00:00, 14187.41it/s]

hello 4242
hello 4243
hello 4244
hello 4245
hello 4246
hello 4247
hello 4248
hello 4249
hello 4250
hello 4251
hello 4252
hello 4253
hello 4254
hello 4255
hello 4256
hello 4257
hello 4258
hello 4259
hello 4260
hello 4261
hello 4262
hello 4263
hello 4264
hello 4265
hello 4266
hello 4267
hello 4268
hello 4269
hello 4270
hello 4271
hello 4272
hello 4273
hello 4274
hello 4275
hello 4276
hello 4277
hello 4278
hello 4279
hello 4280
hello 4281
hello 4282
hello 4283
hello 4284
hello 4285
hello 4286
hello 4287
hello 4288
hello 4289
hello 4290
hello 4291
hello 4292
hello 4293
hello 4294
hello 4295
hello 4296
hello 4297
hello 4298
hello 4299
hello 4300
hello 4301
hello 4302
hello 4303
hello 4304
hello 4305
hello 4306
hello 4307
hello 4308
hello 4309
hello 4310
hello 4311
hello 4312
hello 4313
hello 4314
hello 4315
hello 4316
hello 4317
hello 4318
hello 4319
hello 4320
hello 4321
hello 4322
hello 4323
hello 4324
hello 4325
hello 4326
hello 4327
hello 4328
hello 4329
hello 4330
hello 4331
hello 4332

100%|██████████| 5564/5564 [00:00<00:00, 14189.08it/s]


In [62]:
X

[array([-0.17112084,  0.2430664 ,  0.06151021,  0.08107701,  0.06976219,
        -0.3730621 ,  0.07177719,  0.47848928, -0.16115028, -0.13426207,
        -0.16858625, -0.35098907, -0.11302105,  0.1413097 ,  0.06443638,
        -0.14659056,  0.00854222, -0.32462716, -0.01736764, -0.45920107,
         0.11669326,  0.20288481,  0.1423399 , -0.16421506, -0.1287692 ,
        -0.04220067, -0.30462715, -0.24922961, -0.20311743,  0.09463961,
         0.303661  ,  0.02285371,  0.1291053 , -0.08513179, -0.09721146,
         0.33683947, -0.02267675, -0.18804055, -0.22676672, -0.5323098 ,
         0.01884003, -0.16102266, -0.05631838,  0.01379391,  0.1737561 ,
        -0.10701647, -0.13692488, -0.07310054,  0.12185445,  0.08980919,
         0.18043889, -0.22624382, -0.05652698, -0.00713685, -0.17027327,
         0.0798419 ,  0.19601887, -0.03986557, -0.2673321 ,  0.05100584,
         0.07520174,  0.0939848 , -0.15119824, -0.06541912, -0.3217745 ,
         0.09423964,  0.11650013,  0.22919655, -0.3

In [63]:
len(X)

5564

In [64]:
#independent features
X_new = np.array(X)

In [65]:
X_new.shape

(5564, 100)

In [66]:
X_new[0]

array([-0.17112084,  0.2430664 ,  0.06151021,  0.08107701,  0.06976219,
       -0.3730621 ,  0.07177719,  0.47848928, -0.16115028, -0.13426207,
       -0.16858625, -0.35098907, -0.11302105,  0.14130969,  0.06443638,
       -0.14659056,  0.00854222, -0.32462716, -0.01736764, -0.45920107,
        0.11669326,  0.20288481,  0.1423399 , -0.16421506, -0.1287692 ,
       -0.04220067, -0.30462715, -0.24922961, -0.20311743,  0.09463961,
        0.30366099,  0.02285371,  0.1291053 , -0.08513179, -0.09721146,
        0.33683947, -0.02267675, -0.18804055, -0.22676672, -0.53230977,
        0.01884003, -0.16102266, -0.05631838,  0.01379391,  0.17375609,
       -0.10701647, -0.13692488, -0.07310054,  0.12185445,  0.08980919,
        0.18043889, -0.22624382, -0.05652698, -0.00713685, -0.17027327,
        0.0798419 ,  0.19601887, -0.03986557, -0.26733211,  0.05100584,
        0.07520174,  0.0939848 , -0.15119824, -0.06541912, -0.32177451,
        0.09423964,  0.11650013,  0.22919655, -0.36726955,  0.31

In [67]:
X_new[0].shape

(100,)

In [69]:
#dependent feature
y = pd.get_dummies(messages['label'], drop_first=True).iloc[:, 0].values

y

array([False, False,  True, ..., False, False, False], shape=(5572,))

In [70]:
y.shape

(5572,)

In [ ]:
X_new.shape#clearly we can see that our dependent and independent features have different number of samples
#we need to fix that
#it is because we have some empty messages after preprocessing which resulted in 0 vector and this is due to regular expression removing all words from those messages
#lets find those empty messages and remove them from both X_new and y


(5564, 100)

In [73]:
[ (i, j, k) for i, j, k in zip(map(len, corpus), corpus, messages['message']) if i < 1 ]


[(0, '', 'What you doing?how are you?'),
 (0, '', 'Where @'),
 (0, '', '645'),
 (0, '', 'Can a not?'),
 (0, '', ':) '),
 (0, '', 'What you doing?how are you?'),
 (0, '', ':( but your not here....'),
 (0, '', ':-) :-)')]

In [74]:
#we need to apply conditional filtering to remove those empty messages
y = messages[list(map(lambda x: len(x) > 0, corpus))]
y = pd.get_dummies(y['label'], drop_first=True).iloc[:, 0].values
y.shape

(5564,)

In [77]:
X_new.shape[0]

5564

In [79]:
#final independent feature because we dont have both rows and columns same now
#df=pd.DataFrame()
#for i in range(0,len(X_new)):
#    df=df.append(pd.DataFrame(X_new[i]).reshape(1, -1), ignore_index=True)

In [75]:
#now we have same number of samples in both X_new and y
#model training can be done now
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)


In [80]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [81]:
y_pred = rf.predict(X_test)
y_pred

array([False, False, False, ..., False, False, False], shape=(1113,))

In [82]:
#performance metrics
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.97      0.99      0.98       965
        True       0.94      0.81      0.87       148

    accuracy                           0.97      1113
   macro avg       0.95      0.90      0.93      1113
weighted avg       0.97      0.97      0.97      1113

Accuracy: 0.967654986522911
